In [1]:
#kerasをインポートする
from keras import backend
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense,Activation,LSTM
from keras.utils import np_utils
from keras.models import load_model    
from keras.layers.core import Dropout

#pandasとnumpyをインポートする
import pandas as pd
import numpy as np

#すべての警告を非表示にする
import warnings
warnings.filterwarnings('ignore')

#Word2Vecをインポートする
from gensim.models.word2vec import Word2Vec

#MeCab:形態素解析器のインポート
import MeCab

In [2]:
class LstmClass:
    def __init__(self):
        #中間総数
        self.n_hidden = 32
        #入力データ数
        self.data_dim = 50
        self.timesteps = 30
        #出力データ数
        self.num_classes = 6
        
    def lstmModel(self):
        #LSTMの設定
        lstm_model = Sequential()
        lstm_model.add(LSTM(self.n_hidden, return_sequences=True,
        input_shape=(self.timesteps, self.data_dim)))  # 32次元のベクトルのsequenceを出力する

        lstm_model.add(LSTM(self.n_hidden))  # 32次元のベクトルを一つ出力する
        lstm_model.add(Dense(self.num_classes, activation='softmax'))

        lstm_model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        
        return lstm_model

    def morphologicAlanalysis (self):
        words_arr2 = []
        
        for suvey_row in range(1):
            input_text = input()
            text = input_text
            #形態素解析の実行
            tagger = MeCab.Tagger()
            words = tagger.parse(text).splitlines()

            words_arr = []
            for i in range(self.timesteps):
                if(i < len(words)):
                    word_tmp = words[i].split()[0]
                    words_arr.append(word_tmp)
                else:
                    word_tmp = 'A'
                    words_arr.append(word_tmp)
        
        return words_arr

    def lstmProb(self,lstm_model,words_arr):
        #LSTMの重さをロード
        lstm_model.load_weights('w2v_lstm_weights.h5')
        #テストデータの予測を行う
        model_path = './word2vec.gensim.model'
        w2v_model = Word2Vec.load(model_path)
        #予測を行う
        x_test =[]
        x_test.append(w2v_model[words_arr])
        x_test = np.array(x_test).astype(np.float32)
        prob = lstm_model.predict(x_test)
        
        return prob
        
    def resultView(self,prob):
        #結果を表示する
        print("このアンケートの顧客満足度は",max(prob[0]),'の確率で',np.argmax(prob[0]),"です。")

In [3]:
def main():
    #LSTMのモデルを決定
    lstm = LstmClass()
    #LSTMによる予測を行う
    prob = lstm.lstmProb(lstm.lstmModel(),lstm.morphologicAlanalysis())
    #結果を表示する
    lstm.resultView(prob)

if __name__ == '__main__':
    main()

スポーツできる場所があるのが良い
このアンケートの顧客満足度は 0.9938862 の確率で 5 です。
